In [1]:
import pandas
from geosnap import Community
from geosnap import datasets
import fiona
import geopandas
%matplotlib inline
import matplotlib.pyplot as plt
import segregation

# questions for today 5/27
# bucket problem with geosnap?
# losing district and county name with frpm aggregation
# merge operation breaking ela currstatus 

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\geosnap\_data.py:123: UserWarning: Unable to locate local census data. Streaming instead.
If you plan to use census data repeatedly you can store it locally with the io.store_census function for better performance
  "Unable to locate local census data. Streaming instead.\n"
Loading manifest: 100%|██████████| 5/5 [00:00<00:00, 5009.92entries/s]
C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.cluster.spectral module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_column', 500)

In [3]:
ele = geopandas.read_file("districts_tiger/tl_2015_us_sde.shp")
# Reading in shapefiles from census/tiger

In [4]:
seco = geopandas.read_file("districts_tiger/tl_2015_us_sds.shp")

In [5]:
uni = geopandas.read_file("districts_tiger/tl_2015_us_sdu.shp")

In [6]:
ele = ele.loc[ele['STATEFP'] == '06']
# slicing out California
# elementary school districts in California

In [7]:
seco = seco.loc[seco['STATEFP'] == '06']
# secondary school districts in California

In [8]:
uni = uni.loc[uni['STATEFP'] == '06']
# unified school districts in California

In [9]:
len(ele)+len(seco)+len(uni)

986

## Combining shapefiles into single geodataframe

In [10]:
ele['origin'] = 'elementary'

In [11]:
seco['origin'] = 'secondary'

In [12]:
uni['origin'] = 'unified'

In [13]:
adistricts = uni.append([ele, seco])
#all districts

In [14]:
adistricts = adistricts.sort_values('NAME', ascending=True)

In [15]:
check = adistricts['NAME'].str.contains('County Office')

res = [i for i, val in enumerate(check) if val]
print(res)

[]


In [16]:
adistricts.head(50)

,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,origin,ELSDLEA,SCSDLEA
645,06,01620,0601620,ABC Unified School District,00,KG,12,G5420,None,E,38772821,556042,+33.8637711,-118.0665456,"POLYGON ((-118.10857 33.88775, -118.10843 33.8...",unified,NaN,NaN
29,06,NaN,0601650,Acalanes Union High School District,00,09,12,G5410,None,E,205137702,4413448,+37.8772150,-122.1288576,"POLYGON ((-122.26126 37.92143, -122.26009 37.9...",secondary,NaN,01650
403,06,NaN,0601680,Ackerman Elementary School District,00,KG,08,G5400,None,E,11308882,7628,+38.9366318,-121.0507823,"POLYGON ((-121.07045 38.92923, -121.07045 38.9...",elementary,01680,NaN
658,06,00001,0600001,Acton-Agua Dulce Unified School District,00,KG,12,G5420,None,E,555175128,182565,+34.4509057,-118.2276141,"POLYGON ((-118.38010 34.50369, -118.37724 34.5...",unified,NaN,NaN
376,06,NaN,0601710,Adelanto Elementary School District,00,KG,06,G5400,None,E,868470886,832474,+34.6306380,-117.5205628,"POLYGON ((-117.66785 34.79074, -117.66784 34.7...",elementary,01710,NaN
661,06,01770,0601770,Alameda City Unified School District,00,KG,12,G5420,None,E,27045458,32778370,+37.7419107,-122.2599139,"POLYGON ((-122.34051 37.80166, -122.33262 37.7...",unified,NaN,NaN
703,06,01860,0601860,Albany City Unified School District,00,KG,12,G5420,None,E,4634305,9523833,+37.8906499,-122.3181164,"POLYGON ((-122.37312 37.88388, -122.37114 37.8...",unified,NaN,NaN
488,06,NaN,0601890,Alexander Valley Union Elementary School District,00,KG,12,G5400,None,E,171901135,174852,+38.6860805,-122.7921333,"POLYGON ((-122.86767 38.69496, -122.86211 38.6...",elementary,01890,NaN
88,06,NaN,0606037,Alhambra Unified (9-12) School District,00,09,12,G5410,A,F,11666276,120610,+34.0587656,-118.0964295,"POLYGON ((-118.11681 34.04542, -118.11681 34.0...",secondary,NaN,06037
814,06,00153,0600153,Alhambra Unified School District,00,KG,12,G5420,None,E,30456296,66539,+34.0746675,-118.1376365,"POLYGON ((-118.16935 34.05715, -118.16933 34.0...",unified,NaN,NaN


In [17]:
adsgeo = adistricts['NAME']
# all districs name list

In [18]:
adistricts.shape

(986, 18)

In [20]:
items = pandas.DataFrame(columns=['District Code', 'districtname', 'studentgroup', 'currstatus', 'priorstatus', 'change']) 
items

,District Code,districtname,studentgroup,currstatus,priorstatus,change


In [21]:
adsgeo.unique().shape
# todo: some districts listed twice

(970,)

In [22]:
dupes = adistricts[adistricts['NAME'].isin(adistricts['NAME'].value_counts()[adistricts['NAME'].value_counts()>2].index)]
print(dupes['NAME'].unique())
# 3 Jefferson
# 3 Lakeside
# 3 Pioneer

# create new column indicating countyID

['Jefferson Elementary School District'
 'Lakeside Union Elementary School District'
 'Pioneer Union Elementary School District']


### Harmonizing data

ela + math `currstatus` Average distance from Standard of students who took the Smarter Balanced summative assessment in ela/math


options<br>
* questions of differences across groups? (no geography)
* differences across districts (group by district)
* differences across groups in districts (interaction effects > more pronounced differences in certain districts)

visualization

* what is the best indicator to use for choropleth ()

* or - use state level values as benchmark for each group (how is it reflected in each district, heterogeniety in space?)

## ELA acheivement indicator data
Find the codebook [here](https://www.cde.ca.gov/ta/ac/cm/ela18.asp)

In [93]:
ela = pandas.read_csv('lcff_data/eladownload2018.csv')
ela.shape

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(148838, 32)

In [94]:
ela.head()

,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear
0,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AA,172224,NaN,-51.8,177009.0,NaN,-52.7,0.9,2,3,2,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
1,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AI,16662,NaN,-36.8,17234.0,NaN,-39.1,2.3,2,3,2,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
2,0,X,NaN,NaN,NaN,NaN,NaN,NaN,ALL,3171704,NaN,-6.0,3200441.0,NaN,-8.2,2.2,2,3,2,180,NaN,NaN,NaN,NaN,36140.0,18581.0,51.4,11330.0,31.4,6229.0,17.2,2018
3,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AS,293002,NaN,62.4,295040.0,NaN,60.5,2.0,5,3,5,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
4,0,X,NaN,NaN,NaN,NaN,NaN,NaN,EL,1000703,NaN,-47.1,1021418.0,NaN,-50.4,3.3,2,4,3,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018


## Creating state-level variable

In [95]:
replacements = dict(
    studentgroup = {'ALL':'all_students',
                        'AA':'african_american',
                        'AI':'american_indian',
                        'AS':'asian',
                        'FI':'filipino',
                        'HI':'hispanic',
                        'PI':'pacific_islander',
                        'WH':'white',
                        'MR':'multiple_races',
                        'EL':'english_learners',
                        'ELO':'english_learners_only',
                        'RFP':'rfep_only',
                        'EO':'english_only',
                        'SED':'ses_disad',
                        'SWD':'disabilities',
                        'FOS':'foster_youth',
                        'HOM':'homeless_youth',
                })
# dictionary for replacing acronyms, reuse later

ela['studentgroup'].astype(str)
ela = ela.replace(replacements)
# replace acronyms

j = ela
j = j.loc[j['rtype'] == 'X']
# select by no-type (state)
k = pandas.concat([j.pop(x) for x in ['studentgroup', 
                                      'currstatus', 
                                      'priorstatus', 
                                      'change',
                                      'currdenom']],
                  1)
# slice out columns
ela_state = k.sort_values(by='currstatus', ascending=False)
# state-level ela performance variable

## Aggregation 

In [96]:
# fixing cds codes to match with other datasets
ela['cds'].astype(str)
# changing cds code to string
ela = ela[ela.coe_flag !='Y']
# removing county offices of education
ela = ela.loc[ela['rtype'] == 'D']
# select district level entries
ela['cds'] = ela['cds'].apply(lambda x: '{0:0>14}'.format(x))
# adding leading zero to 13 digit int
ela['District Code'] = [x[2:7] for x in ela['cds']]
# adding district only code
elat = set(ela['cds'])
# creating test set variable
ela.head()

,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear,District Code
164,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,african_american,298,NaN,-43.0,332.0,NaN,-48.7,5.7,2,4,3,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
165,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,american_indian,13,NaN,-14.8,14.0,NaN,14.3,-29.1,2,1,0,0,NaN,NaN,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
166,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,all_students,4550,NaN,36.8,4541.0,NaN,32.8,4.0,4,4,4,90,NaN,NaN,NaN,NaN,38.0,26.0,68.4,6.0,15.8,6.0,15.8,2018,61119
167,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,asian,1288,NaN,48.6,1334.0,NaN,54.0,-5.3,5,2,4,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
168,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,english_learners,1084,NaN,-2.5,1176.0,NaN,-1.4,-1.1,3,3,3,130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119


In [27]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in ela.iterrows(): 
    # Create list for the current row 
    my_list =[rows.studentgroup, rows.currstatus, rows.priorstatus, rows.change, rows.currdenom] 
      
    # append the list to the final list 
    Row_list.append(my_list) 
  
 
Row_list

[['african_american', -43.0, -48.7, 5.7, 298],
 ['american_indian', -14.8, 14.3, -29.1, 13],
 ['all_students', 36.8, 32.8, 4.0, 4550],
 ['asian', 48.6, 54.0, -5.3, 1288],
 ['english_learners', -2.5, -1.4, -1.1, 1084],
 ['english_learners_only', -42.0, -44.7, 2.7, 643],
 ['english_only', 43.5, 38.3, 5.2, 3025],
 ['filipino', 29.8, 16.0, 13.9, 275],
 ['foster_youth', nan, nan, nan, 9],
 ['hispanic', 5.0, 0.9, 4.1, 720],
 ['homeless_youth', -75.5, -56.7, -18.7, 30],
 ['multiple_races', 46.6, 40.4, 6.2, 555],
 ['pacific_islander', -27.3, -49.1, 21.8, 40],
 ['rfep_only', 55.0, 55.5, -0.5, 441],
 ['ses_disad', -13.1, -20.0, 6.9, 1491],
 ['disabilities', -73.1, -80.4, 7.3, 506],
 ['white', 60.8, 54.1, 6.7, 1346],
 ['african_american', -9.6, 12.2, -21.8, 53],
 ['american_indian', nan, nan, nan, 1],
 ['all_students', 57.5, 64.1, -6.6, 1887],
 ['asian', 73.0, 73.7, -0.8, 513],
 ['english_learners', 31.0, 37.6, -6.6, 449],
 ['english_learners_only', -11.4, -12.5, 1.1, 186],
 ['english_only', 60.9

In [97]:
#creating district-level ela performance variable (dictionary)
dfe = ela
dfe = pandas.concat([dfe.pop(x) for x in ['District Code',
                                          'districtname',                                          
                                          'studentgroup', 
                                          'currstatus', 
                                          'priorstatus', 
                                          'change']],
                    1)
# slice out columns

In [99]:
#test1 = pandas.concat([dfe, dfm])
#test1.head()

## Math achievement indicator data
find the codebook [here](https://www.cde.ca.gov/ta/ac/cm/math18.asp)

In [156]:
math = pandas.read_csv('lcff_data/mathdownload2018.csv')
math.shape

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(148705, 32)

In [157]:
math['studentgroup'].astype(str)
math = math.replace(replacements)
# replace codes
p = math
p = p.loc[p['rtype'] == 'X']
# select by no-type (state)
q = pandas.concat([p.pop(x) for x in ['studentgroup', 
                                      'currstatus', 
                                      'priorstatus', 
                                      'change',
                                      'currdenom']],
                  1)
# slice out columns
math_state = q.sort_values(by='currstatus', ascending=False)


## Aggregation

In [158]:
# fixing cds codes to match with other datasets
math['cds'].astype(str)
# changing to string
math = math[math.coe_flag !='Y']
# removing county offices of education
math = math.loc[math['rtype'] == 'D']
# select district level performance
math['cds'] = math['cds'].apply(lambda x: '{0:0>14}'.format(x))
# adding leading zero to 13 digit int
math['District Code'] = [x[2:7] for x in math['cds']]
# adding district only code

In [159]:
math.head()

,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear,District Code
163,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,african_american,297,NaN,-83.9,332.0,NaN,-82.8,-1.1,2,3,2,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
164,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,american_indian,13,NaN,-51.8,14.0,NaN,-50.0,-1.8,2,3,0,0,NaN,NaN,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
165,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,all_students,4543,NaN,14.3,4547.0,NaN,10.8,3.4,4,4,4,90,NaN,NaN,NaN,NaN,38.0,30.0,78.9,6.0,15.8,2.0,5.3,2018,61119
166,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,asian,1287,NaN,43.6,1336.0,NaN,44.8,-1.2,5,3,5,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
167,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,english_learners,1081,NaN,-13.7,1177.0,NaN,-12.9,-0.9,3,3,3,130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119


In [160]:
#creating district-level math performance variable (dictionary)
dfm = math
dfm = pandas.concat([dfm.pop(x) for x in ['District Code',
                                          'districtname',                                          
                                          'studentgroup', 
                                          'currstatus', 
                                          'priorstatus', 
                                          'change']],
                    1)
# slice out columns

In [161]:
dfm.shape

(13603, 6)

In [162]:
dfm

,District Code,districtname,studentgroup,currstatus,priorstatus,change
163,61119,Alameda Unified,african_american,-83.9,-82.8,-1.1
164,61119,Alameda Unified,american_indian,-51.8,-50.0,-1.8
165,61119,Alameda Unified,all_students,14.3,10.8,3.4
166,61119,Alameda Unified,asian,43.6,44.8,-1.2
167,61119,Alameda Unified,english_learners,-13.7,-12.9,-0.9
...,...,...,...,...,...,...
148683,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN
148684,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN
148685,72769,Wheatland Union High,ses_disad,-90.0,-74.0,-16.0
148686,72769,Wheatland Union High,disabilities,-226.6,NaN,NaN


In [122]:
dfmm = dfm
dfm1 = pandas.concat([dfmm.pop(x) for x in ['District Code',
                                           'studentgroup',
                                           'currstatus',
                                           'priorstatus',
                                           'change',
                                          ]])

In [124]:
dfm1

163       61119
164       61119
165       61119
166       61119
167       61119
          ...  
148683      NaN
148684      NaN
148685      -16
148686      NaN
148687     14.7
Length: 68015, dtype: object

# LCFF funding data

[FAQ here](https://www.cde.ca.gov/fg/aa/lc/lcfffaq.asp#CALPADS)

In [125]:
lcff = pandas.read_csv('lcff_data/lcffsnapshot18an.csv')
lcff.shape

(2243, 26)

In [126]:
lcff.columns
# leading AND trailing spaces! inconsistently!! 

Index([' County Code ', 'District Code', 'School Code',
       'Local Educational Agency ', 'Charter Number',
       ' TK/K-3 ADA\nTarget, B-5/B-1 ', ' 4 - 6 ADA\nTarget, B-6/B-2 ',
       ' 7 - 8 ADA\nTarget, B-7/B-3 ', ' 9 - 12 ADA\nTarget, B-8/B-4 ',
       'Unduplicated Pupil Percentage\nTarget, D-1',
       ' Base Grant Funding\nTarget, C-5 ',
       ' Supplemental Grant Funding\nTarget, D-7 ',
       ' Concentration Grant Funding\nTarget, E-8 ',
       ' Necessary Small Schools Allowance\nTarget, F-1 ',
       ' Add-On (Based on 2012-13 Targeted Instructional Improvement Block Grant)\nTarget, F-2 ',
       ' Add-On (Based on 2012-13 Home-to-School Transportation)\nTarget, F-3 ',
       ' Add-On (Based on 2012-13 Small School District Bus Replacement Program)\nTarget, F-4 ',
       ' Total LCFF Target Entitlement\nTarget, G-1/F-1 ',
       ' Total LCFF Floor Entitlement\nTransition, B-13 ',
       ' Current Year Gap Funding (100%)\nTransition, C-3 ',
       ' Economic Recovery Tar

In [127]:
lcff.columns = lcff.columns.str.strip()
# strip leading/trailing spaces

In [128]:
lcff_state_totals = lcff.tail(1)
lcff.drop(lcff.tail(1).index,inplace=True)
# extracting state totals, removing from df

In [129]:
# Slicing by district level cds code
lcff['County Code'] = lcff['County Code'].astype(str)
lcff['District Code'] = lcff['District Code'].astype(int)
lcff['District Code'] = lcff['District Code'].astype(str)
lcff['School Code'] = lcff['School Code'].astype(int)
lcff['School Code'] = lcff['School Code'].astype(str)
# Changing to strings
lcff['School Code'] = lcff['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# Front fill school codes to == 7
lcff = lcff.loc[lcff['School Code'] == '0000000']
# select districts, district entries do not have a school level cds code
lcff['cds'] = lcff['County Code'] + lcff['District Code'] + lcff['School Code']
# Combining into cds code
col_name='cds'
first_col = lcff.pop(col_name)
lcff.insert(0, col_name, first_col)
# Moving to front 
lcff.drop('County Code', axis=1, inplace=True)
# lcff.drop('District Code', axis=1, inplace=True)
lcff.drop('School Code', axis=1, inplace=True)
len(lcff)

944

In [130]:
# renaming important columns
lcff = lcff.rename(columns={'Charter Number': 'charter_number',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_percentage',
                     'Base Grant Funding\nTarget, C-5': 'base_grant',
                     'Supplemental Grant Funding\nTarget, D-7': 'supplemental_grant',
                     'Concentration Grant Funding\nTarget, E-8': 'concentration_grant',
                     'Total LCFF Target Entitlement\nTarget, G-1/F-1': 'total_grants',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_count',
                     'Necessary Small Schools Allowance\nTarget, F-1': 'nec_small_schools'
                           })

In [131]:
lcff.head()
# cds codes are now concurrent with documentation and comparable to acheivement data

,cds,District Code,Local Educational Agency,charter_number,"TK/K-3 ADA\nTarget, B-5/B-1","4 - 6 ADA\nTarget, B-6/B-2","7 - 8 ADA\nTarget, B-7/B-3","9 - 12 ADA\nTarget, B-8/B-4",unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,nec_small_schools,"Add-On (Based on 2012-13 Targeted Instructional Improvement Block Grant)\nTarget, F-2","Add-On (Based on 2012-13 Home-to-School Transportation)\nTarget, F-3","Add-On (Based on 2012-13 Small School District Bus Replacement Program)\nTarget, F-4",total_grants,"Total LCFF Floor Entitlement\nTransition, B-13","Current Year Gap Funding (100%)\nTransition, C-3","Economic Recovery Target\nTransition, D-1","Miscellaneous Adjustments\nTransition, E-1","Total Local Revenue or In-Lieu of Property Taxes\nTransition, F-2/F-7","Education Protection Account Entitlement\nTransition, F-5/F-9","Net State Aid\nTransition, F-6/F-10","Additional SA for MSA Guarantee\nTransition, H-1/H-2"
10,01611190000000,61119,Alameda Unified,NaN,"2,870.43","1,917.91","1,219.66","3,050.17",0.3486,"75,938,983","5,294,466",-,-,"665,778","258,889",-,"82,158,116","77,176,229","4,981,887",-,-,"29,277,891","14,772,333","38,107,892",-
16,01611270000000,61127,Albany City Unified,NaN,"1,060.09",806.37,554.25,"1,131.64",0.2833,"29,644,972","1,679,684",-,-,-,"27,884",-,"31,352,540","29,460,891","1,891,649",-,-,"11,005,115","5,777,202","14,570,223",-
17,01611430000000,61143,Berkeley Unified,NaN,"2,828.54","2,101.08","1,397.55","3,070.55",0.3519,"78,556,532","5,528,808",-,-,"4,293,233","991,048",-,"89,369,621","84,726,528","4,643,093","820,253",-,"43,425,650","8,875,387","37,888,837",-
19,01611500000000,61150,Castro Valley Unified,NaN,"2,617.44","2,032.50","1,400.40","2,918.48",0.2904,"74,911,585","4,350,864",-,-,-,"292,158",-,"79,554,607","74,489,641","5,064,966",-,-,"26,572,525","14,660,230","38,321,852",-
20,01611680000000,61168,Emery Unified,NaN,237.65,151.54,104.99,191.97,0.7953,"5,702,229","906,997","699,378",-,-,"18,271",-,"7,326,875","6,698,947","627,928","108,723",-,"5,582,328","137,230","1,716,040",-


In [132]:
#is this done?
dflcff = lcff[['District Code', 'Local Educational Agency', 'unduplicated_pupil_count', 'base_grant', 'supplemental_grant', 'concentration_grant', 'total_grants', 'cds' ]].copy()
# dflcff.set_index(lcff['District Code'])

# Free and reduced meal plan data
Used by policy makers as a stand-in for a  segregation indicator. 

Find the codebook [here](https://www.cde.ca.gov/ds/sd/sd/fsspfrpm.asp).

In [133]:
frpm = pandas.read_csv('lcff_data/frpm1819.csv', encoding = "ISO-8859-1", skiprows=1)
frpm.shape
# aggregate figures by district
# jump down to 'mealagg' cell

(10520, 28)

In [134]:
# renaming important columns
frpm = frpm.rename(columns={'Percent (%) \nEligible FRPM \n(K-12)': 'percent_eligible_k12_frpm', 
                            'Percent (%) \nEligible Free \n(K-12)': 'percent_eligible_k12_free', 
                            'Percent (%) \nEligible FRPM \n(Ages 5-17)' : 'percent_eligible_by_age',
                            'Enrollment \n(Ages 5-17)': 'enrollment_by_age',
                            'FRPM Count \n(Ages 5-17)': 'frpm_count_by_age',
                            'Free Meal \nCount \n(Ages 5-17)': 'free_count_by_age',
                            'District Type': 'district_type',
                            'District Name': 'district_name',
                            'County Name': 'county_name',
                            
                           })

In [135]:
frpm.head()

,Academic Year,County Code,District Code,School Code,county_name,district_name,School Name,district_type,School Type,Educational \nOption Type,NSLP \nProvision \nStatus,Charter \nSchool \n(Y/N),Charter \nSchool \nNumber,Charter \nFunding \nType,IRC,Low Grade,High Grade,Enrollment \n(K-12),Free Meal \nCount \n(K-12),percent_eligible_k12_free,FRPM Count \n(K-12),percent_eligible_k12_frpm,enrollment_by_age,free_count_by_age,Percent (%) \nEligible Free \n(Ages 5-17),frpm_count_by_age,percent_eligible_by_age,CALPADS Fall 1 \nCertification Status
0,2018-2019,1,10017,112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,County Office of Education (COE),High Schools (Public),Traditional,NaN,Yes,0811,Directly funded,Y,9,12,385,262,68.1%,306,79.5%,370,249,67.3%,291,78.6%,Y
1,2018-2019,1,10017,123968,Alameda,Alameda County Office of Education,Community School for Creative Education,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1284,Directly funded,Y,K,8,241,118,49.0%,120,49.8%,224,113,50.4%,115,51.3%,Y
2,2018-2019,1,10017,124172,Alameda,Alameda County Office of Education,Yu Ming Charter,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1296,Directly funded,Y,K,8,445,58,13.0%,64,14.4%,445,58,13.0%,64,14.4%,Y
3,2018-2019,1,10017,125567,Alameda,Alameda County Office of Education,Urban Montessori Charter,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1383,Directly funded,Y,K,8,432,113,26.2%,131,30.3%,418,111,26.6%,129,30.9%,Y
4,2018-2019,1,10017,130401,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,County Office of Education (COE),Juvenile Court Schools,Juvenile Court School,NaN,No,NaN,NaN,N,K,12,53,53,100.0%,53,100.0%,50,50,100.0%,50,100.0%,Y


In [136]:
#creating CDS code
frpm['County Code'] = frpm['County Code'].astype(str)
frpm['County Code'] = frpm['County Code'].apply(lambda x: '{0:0>2}'.format(x))
frpm['District Code'] = frpm['District Code'].astype(str)
frpm['School Code'] = frpm['School Code'].astype(str)
frpm['School Code'] = frpm['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# converting to strings, adding leading zeroes 
frpm['cds'] = frpm['County Code'] + frpm['District Code'] + frpm['School Code']
# combining codes

frpm = frpm[frpm.district_type !='County Office of Education (COE)']
# removing county offices of education

col_name1='cds'
first_col1 = frpm.pop(col_name1)
frpm.insert(0, col_name1, first_col1)
# Moving to front

frpm.drop('County Code', axis=1, inplace=True)
# frpm.drop('District Code', axis=1, inplace=True)
frpm.drop('School Code', axis=1, inplace=True)
# dropping split codes

In [137]:
frpm.columns

Index(['cds', 'Academic Year', 'District Code', 'county_name', 'district_name',
       'School Name', 'district_type', 'School Type',
       'Educational \nOption Type', 'NSLP \nProvision \nStatus',
       'Charter \nSchool \n(Y/N)', 'Charter \nSchool \nNumber',
       'Charter \nFunding \nType', 'IRC', 'Low Grade', 'High Grade',
       'Enrollment \n(K-12)', 'Free Meal \nCount \n(K-12)',
       'percent_eligible_k12_free', 'FRPM Count \n(K-12)',
       'percent_eligible_k12_frpm', 'enrollment_by_age', 'free_count_by_age',
       'Percent (%) \nEligible Free \n(Ages 5-17)', 'frpm_count_by_age',
       'percent_eligible_by_age', 'CALPADS Fall 1 \nCertification Status'],
      dtype='object')

In [138]:
# aggregate all schools into district level

frpm['enrollment_by_age'] = frpm['enrollment_by_age'].str.replace(',', '').astype(int)
frpm['frpm_count_by_age'] = frpm['frpm_count_by_age'].str.replace(',', '').astype(int)
frpm['free_count_by_age'] = frpm['free_count_by_age'].str.replace(',', '').astype(int)
# striping commas, converting to int 

In [139]:
frpm.columns

Index(['cds', 'Academic Year', 'District Code', 'county_name', 'district_name',
       'School Name', 'district_type', 'School Type',
       'Educational \nOption Type', 'NSLP \nProvision \nStatus',
       'Charter \nSchool \n(Y/N)', 'Charter \nSchool \nNumber',
       'Charter \nFunding \nType', 'IRC', 'Low Grade', 'High Grade',
       'Enrollment \n(K-12)', 'Free Meal \nCount \n(K-12)',
       'percent_eligible_k12_free', 'FRPM Count \n(K-12)',
       'percent_eligible_k12_frpm', 'enrollment_by_age', 'free_count_by_age',
       'Percent (%) \nEligible Free \n(Ages 5-17)', 'frpm_count_by_age',
       'percent_eligible_by_age', 'CALPADS Fall 1 \nCertification Status'],
      dtype='object')

In [140]:
names = frpm['district_name'].unique()
len(names)

966

In [141]:
codes = []
for name in names:
    temp = frpm[frpm['district_name'] == name]
    codes.append(temp['District Code'].values[0])

In [142]:
len(codes)

966

In [143]:
t5 = pandas.DataFrame(list(zip(names, codes)), 
               columns =['district_name', 'District Code'])

# some names are getting the same codes?

In [144]:
t5

,district_name,District Code
0,California School for the Blind (State Special...,31609
1,California School for the Deaf-Fremont (State ...,31617
2,Alameda Unified,61119
3,Albany City Unified,61127
4,Berkeley Unified,61143
...,...,...
961,Camptonville Elementary,72728
962,Marysville Joint Unified,72736
963,Plumas Lake Elementary,72744
964,Wheatland,72751


In [145]:
# If nothing is int (and everything is object), all columns live through the groupby
# if we convert to int before groupby (to make .sum() work), only int lives through groupby

dffrpm = frpm.groupby(by='district_name').sum()
# aggregating

In [146]:
dffrpm.columns

Index(['enrollment_by_age', 'free_count_by_age', 'frpm_count_by_age'], dtype='object')

In [147]:
dffrpm['pct_frpm_by_age'] = dffrpm.frpm_count_by_age / dffrpm.enrollment_by_age
dffrpm['pct_free_by_age'] = dffrpm.free_count_by_age / dffrpm.enrollment_by_age
# inferring more convinient than copying lul

In [148]:
dffrpm.shape
# need to include district name and county name in the aggreagtion
# why do we lose columns in the groupby operation? dtypes object preventing groupby?

(966, 5)

In [149]:
t5.shape

(966, 2)

In [150]:
dffrpm = pandas.merge(dffrpm, t5, how='left', on=['district_name'])
dffrpm

,district_name,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,District Code
0,ABC Unified,20022,8508,9994,0.499151,0.424933,64212
1,Acalanes Union High,5487,334,358,0.065245,0.060871,61630
2,Ackerman Charter,584,154,180,0.308219,0.263699,66761
3,Acton-Agua Dulce Unified,13553,5745,6389,0.471409,0.423891,75309
4,Adelanto Elementary,8512,6158,6708,0.788064,0.723449,67587
...,...,...,...,...,...,...,...
961,Yosemite Unified,2036,881,977,0.479862,0.432711,76414
962,Yreka Union Elementary,984,634,702,0.713415,0.644309,70508
963,Yreka Union High,610,269,290,0.475410,0.440984,70516
964,Yuba City Unified,12839,8182,9518,0.741335,0.637277,71464


## Test Variables for merge with District Codes

In [151]:
geot = set(adsgeo)
len(geot)
# There are no district codes in the geometries - must match by district name. 

970

In [152]:
frpmt = set(dffrpm['District Code'])
len(frpmt)

966

In [153]:
lcfft = set(dflcff['District Code'])
len(lcfft)

944

In [154]:
elat = set(dfe['District Code'])
len(elat)

932

In [163]:
matht = set(dfm['District Code'])
len(matht)
# lowest common denominator

931

In [164]:
d2 = lcfft.difference(matht)
d2

{'61994',
 '63933',
 '66761',
 '68627',
 '70318',
 '70680',
 '70714',
 '70730',
 '70797',
 '70839',
 '70854',
 '70862',
 '70912',
 '70920',
 '71274',
 '71670'}

In [165]:
d3 = lcfft.difference(elat)
d3

{'61994',
 '63933',
 '66761',
 '68627',
 '70680',
 '70714',
 '70730',
 '70797',
 '70839',
 '70854',
 '70862',
 '70912',
 '70920',
 '71274',
 '71670'}

In [166]:
drops = d2.intersection(d3)
len(drops)

15

In [167]:
d2.difference(drops)

{'70318'}

In [168]:
drops = d2
len(drops)

16

In [169]:
d4 = frpmt.difference(matht)
len(d4)

49

In [170]:
d2.difference(d4)

set()

In [171]:
d3.difference(d4)

set()

### d4 contains the list of districts to drop to merge all 4 datasets. 

In [172]:
len(d4)

49

In [173]:
d4

{'31609',
 '31617',
 '31625',
 '32276',
 '61994',
 '63933',
 '66761',
 '68627',
 '70318',
 '70680',
 '70714',
 '70730',
 '70797',
 '70839',
 '70854',
 '70862',
 '70912',
 '70920',
 '71274',
 '71670',
 '75630',
 '75663',
 '75697',
 '76547',
 '76885',
 '76893',
 '76901',
 '76919',
 '76927',
 '76943',
 '76950',
 '76968',
 '76992',
 '77024',
 '77032',
 '77065',
 '77081',
 '77099',
 '77107',
 '77131',
 '77149',
 '77156',
 '77164',
 '77172',
 '77180',
 '77198',
 '77206',
 '77214',
 '77222'}

# Examining data

### State-level performance data

In [90]:
ela_state

,studentgroup,currstatus,priorstatus,change,currdenom
3,asian,62.4,60.5,2.0,293002
7,filipino,44.0,41.2,2.7,71139
11,multiple_races,28.6,26.3,2.3,118753
16,white,27.7,26.9,0.8,734863
6,english_only,6.5,4.6,1.8,1795002
13,rfep_only,2.2,-0.4,2.7,488023
2,all_students,-6.0,-8.2,2.2,3171704
12,pacific_islander,-21.3,-22.0,0.7,14894
9,hispanic,-31.3,-34.5,3.2,1738615
14,ses_disad,-34.7,-38.7,4.0,1982522


how to loop?

```for score in student group
    select student group, return averaged score```

In [91]:
# both english learning groups
(-47.1 + -94.1) / 2

-70.6

In [92]:
# combine `studentgroups`
# lcff_group = 'ses_disad', 'english_learners', 'homeless_youth', 'foster_youth' 'english_learners_only'

d = (-34.7 + -47.1 + -51.0 + -79.2 + -94.1)
d / 5

-61.220000000000006

In [93]:
math_state

,studentgroup,currstatus,priorstatus,change,currdenom
3,asian,56.7,54.3,2.4,293068
7,filipino,13.1,10.3,2.8,71096
11,multiple_races,1.9,0.1,1.9,118473
16,white,-1.0,-1.6,0.6,733213
6,english_only,-25.2,-26.3,1.1,1790583
13,rfep_only,-29.7,-31.1,1.4,487555
2,all_students,-36.4,-37.7,1.3,3166345
12,pacific_islander,-52.0,-51.3,-0.7,14849
9,hispanic,-65.8,-67.5,1.7,1735982
14,ses_disad,-67.4,-69.7,2.3,1978909


In [101]:
dfe = dfe.rename(columns={'currstatus': 'ela_currstatus',
                          'priorstatus': 'ela_priorstatus',
                          'change' : 'ela_change'
                         })
dfe
# dataframe english

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7
165,61119,Alameda Unified,american_indian,-14.8,14.3,-29.1
166,61119,Alameda Unified,all_students,36.8,32.8,4.0
167,61119,Alameda Unified,asian,48.6,54.0,-5.3
168,61119,Alameda Unified,english_learners,-2.5,-1.4,-1.1
...,...,...,...,...,...,...
148816,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN
148817,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN
148818,72769,Wheatland Union High,ses_disad,-28.4,-7.9,-20.6
148819,72769,Wheatland Union High,disabilities,-150.3,NaN,NaN


In [102]:
dfm = dfm.rename(columns={'currstatus': 'm_currstatus',
                          'priorstatus': 'm_priorstatus',
                          'change' : 'm_change'
                         })

dfm
# dataframe math

,District Code,districtname,studentgroup,m_currstatus,m_priorstatus,m_change
163,61119,Alameda Unified,african_american,-83.9,-82.8,-1.1
164,61119,Alameda Unified,american_indian,-51.8,-50.0,-1.8
165,61119,Alameda Unified,all_students,14.3,10.8,3.4
166,61119,Alameda Unified,asian,43.6,44.8,-1.2
167,61119,Alameda Unified,english_learners,-13.7,-12.9,-0.9
...,...,...,...,...,...,...
148683,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN
148684,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN
148685,72769,Wheatland Union High,ses_disad,-90.0,-74.0,-16.0
148686,72769,Wheatland Union High,disabilities,-226.6,NaN,NaN


In [ ]:
test = pandas.concat([dfm, dfe], axis=1)

In [96]:
dflcff
# dataframe lcff

,District Code,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds
10,61119,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
16,61127,Albany City Unified,0.2833,"29,644,972","1,679,684",-,"31,352,540",01611270000000
17,61143,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,"89,369,621",01611430000000
19,61150,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,"79,554,607",01611500000000
20,61168,Emery Unified,0.7953,"5,702,229","906,997","699,378","7,326,875",01611680000000
...,...,...,...,...,...,...,...,...
2233,72728,Camptonville Elementary,0.6988,"420,692","58,796","31,299","537,244",58727280000000
2235,72736,Marysville Joint Unified,0.8168,"76,193,862","12,447,028","10,164,261","100,261,913",58727360000000
2238,72744,Plumas Lake Elementary,0.3868,"10,261,440","793,825",-,"11,108,901",58727440000000
2239,72751,Wheatland,0.4783,"9,947,133","951,543",-,"11,234,349",58727510000000


In [97]:
s1 = pandas.merge(dfe, dfm, how='left', on=['District Code'])
s1.drop('districtname_y', axis=1, inplace=True)
s1.drop('studentgroup_y', axis=1, inplace=True)
s1 = s1.rename(columns={'districtname_x': 'districtname',
                          'studentgroup_x': 'studentgroup',
                        })


In [98]:
s1.head(200)
# english groups got messed up here

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change,m_currstatus,m_priorstatus,m_change
0,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-83.9,-82.8,-1.1
1,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-51.8,-50.0,-1.8
2,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,14.3,10.8,3.4
3,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,43.6,44.8,-1.2
4,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-13.7,-12.9,-0.9
5,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-42.4,-42.9,0.5
6,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,17.8,13.0,4.7
7,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-3.5,-12.7,9.3
8,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,NaN,NaN,NaN
9,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-24.4,-26.7,2.3


In [99]:
s2 = pandas.merge(s1, dflcff, how='left', on=['District Code'])
s2
# probably a better way to do this

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change,m_currstatus,m_priorstatus,m_change,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds
0,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-83.9,-82.8,-1.1,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
1,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-51.8,-50.0,-1.8,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
2,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,14.3,10.8,3.4,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
3,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,43.6,44.8,-1.2,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
4,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-13.7,-12.9,-0.9,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207554,72769,Wheatland Union High,white,11.7,14.2,-2.5,NaN,NaN,NaN,Wheatland Union High,0.4283,"7,062,422","604,967",-,"7,852,628",58727690000000
207555,72769,Wheatland Union High,white,11.7,14.2,-2.5,NaN,NaN,NaN,Wheatland Union High,0.4283,"7,062,422","604,967",-,"7,852,628",58727690000000
207556,72769,Wheatland Union High,white,11.7,14.2,-2.5,-90.0,-74.0,-16.0,Wheatland Union High,0.4283,"7,062,422","604,967",-,"7,852,628",58727690000000
207557,72769,Wheatland Union High,white,11.7,14.2,-2.5,-226.6,NaN,NaN,Wheatland Union High,0.4283,"7,062,422","604,967",-,"7,852,628",58727690000000


### play cells

In [24]:
i = ela.loc[ela['rtype'] == 'S']
o = i = pandas.concat([i.pop(x) for x in ['schoolname', 
                                      'cds']],
                  1)
o
# had to check this school's cds code to validate with another dataset

,schoolname,cds
30,Envision Academy for Arts & Technology,1100170112607
31,Envision Academy for Arts & Technology,1100170112607
32,Envision Academy for Arts & Technology,1100170112607
33,Envision Academy for Arts & Technology,1100170112607
34,Envision Academy for Arts & Technology,1100170112607
...,...,...
148833,Wheatland Union High,58727695838305
148834,Wheatland Union High,58727695838305
148835,Wheatland Union High,58727695838305
148836,Wheatland Union High,58727695838305


# Census Data

#### SCAG counties and their FIPS codes:
<br> Los Angeles (06037)<br> Orange (06059)<br> Riverside (06065)<br> San Bernardino (06071)<br> San Diego (06073)<br>Imperial (06025)<br> Ventura (06111)

In [2]:
cacom = Community.from_census(state_fips='06')

NameError: name 'Community' is not defined

In [ ]:
cacom.gdf['county'] = cacom.gdf.geoid.str[:5]
# create counties variable

In [ ]:
cacom.gdf.groupby('county').sum()
#group

In [26]:
scag = Community.from_census(county_fips=['06025', '06037', '06059', '06071', '06111', '06073', '06065'])

NameError: name 'Community' is not defined

In [27]:
rivco = Community.from_census(county_fips=['06065'])

NameError: name 'Community' is not defined

In [28]:
orangeco = Community.from_census(county_fips=['06059'])

NameError: name 'Community' is not defined

In [ ]:
laco = Community.from_census(county_fips=['06037'])

In [ ]:
sbco = Community.from_census(county_fips=['06071'])

In [ ]:
sdco = Community.from_census(county_fips=['06073'])

In [ ]:
f, ax = plt.subplots(1, figsize=(30, 30))
ax = rivco.gdf[rivco.gdf.year==2010].plot('p_poverty_rate_over_65', axes=ax)
plt.show()

# change gdf, variable, and year

### This 'plot' function will produce three maps, one from each of the census data years. We can use it to pass in communities and a specific variable to provide a preliminary view of changes over 30 years.

In [ ]:
def plot(community, column):
    fig, axs = plt.subplots(1,3, figsize=(20,20))
    axs=axs.flatten()
    
    community.gdf[community.gdf.year==1990].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[0])
    axs[0].axis('off')
    axs[0].set_title('1990')

    community.gdf[community.gdf.year==2000].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[1])
    axs[1].axis('off')
    axs[1].set_title('2000')

    community.gdf[community.gdf.year==2010].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[2])
    axs[2].axis('off')
    axs[2].set_title('2010')

In [ ]:
datasets.codebook()

### Variables of interest from census:
#### economic indicators
median_household_income <br>
median_home_value OR median_contract_rent<br>
median_income_whitehh<br>
median_income_blackhh<br>
median_income_hispanichh<br>
n_edu_college_greater - persons with at least 4-year degree (also available as %)<br>
n_edu_hs_less - persons with less than high school (also available as %)<br>

#### demographic indicators
n_recent_immigrant_pop (within the last 10 years)<br>
n_limited_english OR n_other_language<br>
n_hispanic_persons<br>
n_black_persons<br>
n_white_persons

In [ ]:
from segregation.aspatial import GiniSeg
index = GiniSeg(scag.gdf, 'n_mexican_pop', 'n_total_pop')
type(index)

# MERGE SCHOOL DISTRICTS WITH CENSUS DATA

## Alternative community construction

Can also construct communities from longitudinal tract database (ltdb)

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('Riverside')]

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('Orange')]

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('San')]

In [ ]:
LA = Community.from_ltdb(msa_fips='31080')

In [ ]:
SD = Community.from_ltdb(msa_fips='41740')

In [ ]:
riverside = Community.from_ltdb(msa_fips='40140')